IMPLEMENTACION DE UN MODELO MULTILABEL CON REDES NEURONALES EN KERAS

In [1]:
# Import libraries
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import os
import numpy as np

import tensorflow as tf

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential, load_model

In [ ]:
#Primero cargamos los datos

In [3]:
df_train = pd.read_csv("entrenamiento.csv",sep=";")

In [5]:
del df_train["Unnamed: 0"]

In [6]:
df_train.head() # visualizamos los datos

,fecha_dato,ncodpers,age,antiguedad,nomprov,ind_actividad_cliente,renta,ind_ahor_fin_ult1,ind_aval_fin_ult1,ind_cco_fin_ult1,...,cod_prov_37.0,cod_prov_41.0,cod_prov_45.0,cod_prov_46.0,cod_prov_47.0,cod_prov_48.0,cod_prov_50.0,segmento_01 - TOP,segmento_02 - PARTICULARES,segmento_03 - UNIVERSITARIO
0,2015-05-28,1301259,0.5875,0.042969,LIMA,0.0,0.163021,0,0,1,...,0,0,0,0,0,0,0,1,0,0
1,2015-05-28,1374821,0.7250,0.023438,LIMA,0.0,0.064536,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,2015-05-28,1378928,0.4375,0.023438,LIMA,0.0,0.118521,0,0,1,...,0,0,0,0,0,0,0,1,0,0
3,2015-05-28,1254961,0.4375,0.023438,LIMA,1.0,0.364076,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,2015-05-28,759864,0.4625,0.339844,UCAYALI,0.0,0.088534,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [9]:
# seleccionamos los features

In [10]:
cols=['age',
 'antiguedad',
 'renta',
 'sexo_H',
 'sexo_V',
 'canal_entrada_DISPLAY',
 'canal_entrada_EMAIL MARKETING',
 'canal_entrada_MOBILE ADS',
 'canal_entrada_PUBLICIDAD NATIVA',
 'canal_entrada_RETARGETING DINAMICO',
 'canal_entrada_SOCIAL ADS',
 'cod_prov_3.0',
 'cod_prov_8.0',
 'cod_prov_11.0',
 'cod_prov_15.0',
 'cod_prov_18.0',
 'cod_prov_28.0',
 'cod_prov_29.0',
 'cod_prov_30.0',
 'cod_prov_33.0',
 'cod_prov_35.0',
 'cod_prov_36.0',
 'cod_prov_37.0',
 'cod_prov_41.0',
 'cod_prov_45.0',
 'cod_prov_46.0',
 'cod_prov_47.0',
 'cod_prov_48.0',
 'cod_prov_50.0',
 'segmento_01 - TOP',
 'segmento_02 - PARTICULARES',
 'segmento_03 - UNIVERSITARIO']

In [ ]:
#seleccionamos los target

In [12]:
currcols=['ind_ahor_fin_ult1',
 'ind_aval_fin_ult1',
 'ind_cco_fin_ult1',
 'ind_cder_fin_ult1',
 'ind_cno_fin_ult1',
 'ind_ctju_fin_ult1',
 'ind_ctma_fin_ult1',
 'ind_ctop_fin_ult1',
 'ind_ctpp_fin_ult1',
 'ind_deco_fin_ult1',
 'ind_deme_fin_ult1',
 'ind_dela_fin_ult1',
 'ind_ecue_fin_ult1',
 'ind_fond_fin_ult1',
 'ind_hip_fin_ult1',
 'ind_plan_fin_ult1',
 'ind_pres_fin_ult1',
 'ind_reca_fin_ult1',
 'ind_tjcr_fin_ult1',
 'ind_valo_fin_ult1',
 'ind_viv_fin_ult1',
 'ind_nomina_ult1',
 'ind_nom_pens_ult1',
 'ind_recibo_ult1']

In [ ]:
#SELECCIONAMOS EL TRAIN Y TEST

In [14]:
x_train = df_train[cols]
y_train =df_train[currcols]

In [ ]:
#COMENZAMOS CON EL MODELO

In [17]:
model = Sequential() # Define tu red neuronal, para este ejemplo es un NN basico

#Definimos nuestra capa de entrada
model.add(Dense(150, input_dim=x_train.shape[1], activation = 'relu'))
#El primer parametro se refiere a la cantidad de neuronas 

model.add(Dense(24, activation = 'sigmoid')) #Definimos cuantas variables de salida tiene tu modelo, en nuestro ejemplo
#tenemos 24 etiquetas.
#-PARA CASOS MULTILABEL USAMOS LA FUNCION DE ACTIVACION SIGMOID, POR QUE ESTA NOS DA PROBABILIDADES
#INDEPENDIENTES, CUANDO EL CASO SEA UNA MULTICLASE ( MAS DE UNA CLASE EN TU VARIABLE PREDICTORA) ES RECOMENDABLE USAR LA FUNCION DE SOFTMAX,
#LAS PROBABILIDADES SUMAN 1


model.compile(loss = 'binary_crossentropy', optimizer = 'adam') # Compilamos el modelo con una una funcion de perdida "binary_crossentropy",
#es para clasificación binaria de múltiples etiquetas (las etiquetas son independientes) y usaremos categorical_crossentropy cuando
#tengamos una clasificacion multiclase

In [19]:
history=model.fit(x_train.values, y_train.values, validation_split=0.2,epochs=50, batch_size=20) ## Entrenamos nuestro modelo, ingresamos
# la matriz de x e y, definimos la particion de datos que tomara en la validacion cruzada, definimos el "batch_size" que es la cantidad
#de muestras que tomara, la eleccion de un batch size pequeño causara ruido ya que una muestra pequeña no es una buena representacion de los datos
#y uno grande puede causar problemas con la memoria de la instancia que ejecutas, una practica comun es ejecutar el resultado de la raiz cuadrada
# de tu cantidad de datos. Los epochs ejecuta el algoritmo hasta minimizar el error.

Average test loss:  0.19832436591386796


In [ ]:
model.results

In [ ]:
model.save("model_save/" + 'trained_model.h5') # por ultimo guardamos el modelo para que pueda ser consumido en cualquier momento!

In [ ]:
results